In [1]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras    
from tensorflow.keras.layers import Flatten   
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


drive.mount("/content/drive")
path = "/content/drive/MyDrive/Capstone/exercise_datasetV2.csv"
df = pd.read_csv(path)
print(df.head())
banyak_kategori = len(df.index)


Mounted at /content/drive
  Activity, Exercise or Sport (1 hour) Intensity Description  \
0          Cycling, mountain bike, bmx                   NaN   
1  Cycling, <10 mph, leisure bicycling                   NaN   
2             Cycling, >20 mph, racing                   NaN   
3          Cycling, 10-11.9 mph, light                   NaN   
4       Cycling, 12-13.9 mph, moderate                   NaN   

   Duration (minutes)  Calories per kg  
0                  60         1.750730  
1                  60         0.823236  
2                  60         3.294974  
3                  60         1.234853  
4                  60         1.647825  


In [2]:
list_berat = []
for i in range(len(df.index)):
  list_berat.append(1)

df['berat'] =  list_berat
dict_df = {'Activity, Exercise or Sport (1 hour)' : [], 'Duration (minutes)': [], 'Calories per kg': [], 'berat' : []}
df_new = df
for index, row in df.iterrows():
  print(index)
  menit = row['Duration (minutes)']
  activity = row['Activity, Exercise or Sport (1 hour)']
  calories = row['Calories per kg']
  for i in range(1,menit):
    for j in range(2,101):
      new_calories = calories*1.0/60*i*j
      list_activity = dict_df.get('Activity, Exercise or Sport (1 hour)')
      list_duration = dict_df.get('Duration (minutes)')
      list_calories = dict_df.get('Calories per kg')
      list_berat = dict_df.get('berat')
      list_activity.append(activity)
      list_duration.append(i)
      list_calories.append(new_calories)
      list_berat.append(j)
      #new_row = pd.DataFrame({'Activity, Exercise or Sport (1 hour)' : [activity], 'Duration (minutes)': [i], 'Calories per kg': [new_calories], 'berat' : [j]})
      
df_curr = pd.DataFrame(dict_df)   
df_new = pd.concat([df_curr, df_new.loc[:]]).reset_index(drop=True)
#df2 = pd.concat([new_row,df.loc[:]]).reset_index(drop=True)
print(df_new.head())
print(df_new.tail())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
  Activity, Exercise or Sport (1 hour)  Duration (minutes)  Calories per kg  \
0          Cycling, mountain bike, bmx 

In [3]:
print(len(df_new.index))
print(df_new.describe())
print(df_new.dtypes)
df_new.rename(columns = {'Activity, Exercise or Sport (1 hour)':'activity', 'Duration (minutes)' : 'durasi' , 'Calories per kg' : 'calories' }, inplace = True)
print(df_new.head())

1448816
       Duration (minutes)  Calories per kg         berat
count        1.448816e+06     1.448816e+06  1.448816e+06
mean         3.000514e+01     3.467251e+01  5.099144e+01
std          1.703246e+01     3.748635e+01  2.858243e+01
min          1.000000e+00     1.033558e-02  1.000000e+00
25%          1.500000e+01     8.237434e+00  2.600000e+01
50%          3.000000e+01     2.219663e+01  5.100000e+01
75%          4.500000e+01     4.774767e+01  7.600000e+01
max          6.000000e+01     3.644815e+02  1.000000e+02
Activity, Exercise or Sport (1 hour)     object
Duration (minutes)                        int64
Calories per kg                         float64
berat                                     int64
Intensity Description                    object
dtype: object
                      activity  durasi  calories  berat Intensity Description
0  Cycling, mountain bike, bmx       1  0.058358      2                   NaN
1  Cycling, mountain bike, bmx       1  0.087536      3              

In [4]:
target = df['Activity, Exercise or Sport (1 hour)']
print(df_new.head())
numeric_feature_names = ['durasi', 'calories', 'berat']
numeric_features = df_new[numeric_feature_names]
numeric_features.head()

                      activity  durasi  calories  berat Intensity Description
0  Cycling, mountain bike, bmx       1  0.058358      2                   NaN
1  Cycling, mountain bike, bmx       1  0.087536      3                   NaN
2  Cycling, mountain bike, bmx       1  0.116715      4                   NaN
3  Cycling, mountain bike, bmx       1  0.145894      5                   NaN
4  Cycling, mountain bike, bmx       1  0.175073      6                   NaN


,durasi,calories,berat
0,1,0.058358,2
1,1,0.087536,3
2,1,0.116715,4
3,1,0.145894,5
4,1,0.175073,6


In [6]:
jumlah_class = len(df_new['activity'].value_counts())
print(jumlah_class)

248


In [7]:
df_new['activity'] = df_new['activity'].astype('category')
df_new['activity_category'] = df_new['activity'].cat.codes.astype('category')
print(df_new.head())

                      activity  durasi  calories  berat Intensity Description  \
0  Cycling, mountain bike, bmx       1  0.058358      2                   NaN   
1  Cycling, mountain bike, bmx       1  0.087536      3                   NaN   
2  Cycling, mountain bike, bmx       1  0.116715      4                   NaN   
3  Cycling, mountain bike, bmx       1  0.145894      5                   NaN   
4  Cycling, mountain bike, bmx       1  0.175073      6                   NaN   

  activity_category  
0                61  
1                61  
2                61  
3                61  
4                61  


In [8]:
df_new_2 = df_new.drop(columns = ['activity', 'Intensity Description', 'activity_category'])
sc = StandardScaler()
x = pd.DataFrame(sc.fit_transform(df_new_2))

In [9]:
df_new_2['durasi'] = MinMaxScaler().fit_transform(np.array(df_new_2['durasi']).reshape(-1,1))
df_new_2['calories'] = MinMaxScaler().fit_transform(np.array(df_new_2['calories']).reshape(-1,1))
df_new_2['berat'] = MinMaxScaler().fit_transform(np.array(df_new_2['berat']).reshape(-1,1))

In [10]:
y = tf.keras.utils.to_categorical(df_new["activity_category"].values, num_classes=jumlah_class)
print(x.values)
print(type(x.values))
print(y)
print(type(y))
y = np.array(df_new['activity_category'])
print(y)
x_train, x_test, y_train, y_test = train_test_split(x.values, y, test_size=0.2)

[[-1.70293372 -0.9233803  -1.71404105]
 [-1.70293372 -0.92260191 -1.67905451]
 [-1.70293372 -0.92182353 -1.64406797]
 ...
 [ 1.76104218 -0.90846636 -1.74902759]
 [ 1.76104218 -0.90297612 -1.74902759]
 [ 1.76104218 -0.90297612 -1.74902759]]
<class 'numpy.ndarray'>
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<class 'numpy.ndarray'>
[ 61  61  61 ... 207 232 208]


In [11]:
print(x_train)
print(y_train)
print(x_test)
print(y_test)

[[ 1.35006199  0.14199908  0.1402456 ]
 [-0.05901295 -0.434158    0.17523214]
 [-0.41128169 -0.51427182 -1.3291891 ]
 ...
 [-0.05901295 -0.35971712  0.70003025]
 [ 0.76294743  1.24878701  0.63005717]
 [-0.99839625  0.03741788  0.9099495 ]]
[ 25 223 156 ...  95  24 192]
[[-1.23324207 -0.57547081  0.07027252]
 [-0.35257023 -0.78427141 -1.50412181]
 [ 1.05650471  0.4454262   0.94493604]
 ...
 [ 1.70233072  1.36600839  0.07027252]
 [-0.29385878  0.28291517  1.29480144]
 [-0.05901295 -0.71901132 -0.69943137]]
[ 87 210  72 ... 148 122 116]


In [12]:
x, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=7)
print(x)
print(y)

[[ 0.2929949  -4.21223056 -1.288332   ... -4.43170633 -2.82646737
   0.44916808]
 [-0.06839901  5.51884147 11.2389773  ... -3.08994781  1.19029898
   1.62025622]
 [ 0.73161622 -0.68468633 -0.98174194 ...  5.65429655 -0.64659866
  -3.15652999]
 ...
 [ 0.81230832  0.29333773  3.55727154 ...  7.52278375 -4.50067701
  -1.92525878]
 [ 2.62760166 -1.9607565  -7.1050466  ...  0.02433393 -0.77573778
   4.04660465]
 [-0.97292653  0.76166769  3.98307684 ...  0.85864477  2.406057
   2.33338943]]
[1 1 1 0 0 0 1 1 0 1 0 1 1 0 1 1 1 1 0 0 1 0 0 1 1 0 1 0 1 1 1 0 1 0 0 1 0
 1 0 1 0 1 1 1 0 1 0 0 1 0 1 1 1 1 1 1 1 0 0 1 1 1 0 0 1 0 0 1 0 1 1 1 0 0
 0 1 0 1 0 0 1 1 0 1 0 1 1 1 0 0 1 0 1 0 1 1 0 0 1 1 1 0 1 1 0 0 1 1 0 1 0
 0 0 1 1 1 1 0 0 0 1 0 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 1 0 1 1 0 1 0
 1 1 1 0 1 1 0 1 1 0 1 0 0 0 0 0 0 1 0 1 1 1 0 1 1 1 1 0 0 0 0 0 0 1 0 0 0
 0 0 1 1 1 0 0 0 1 0 0 1 0 1 1 1 1 1 0 0 0 1 1 0 1 0 1 1 0 0 1 1 0 0 1 1 1
 0 0 1 1 0 1 1 1 0 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 1 1 1 0 0

In [ ]:
model = GradientBoostingClassifier(n_estimators = banyak_kategori)
model.fit(x_train, y_train)

In [ ]:
y_pred = model.predict(x_test)
y_test_class = np.argmax(y_test, axis=1)
classes_pred = np.argmax(y_pred,axis=1)
report = classification_report(y_test_class, classes_pred, output_dict=True, zero_division=0)
print(report)

# Extract the metrics
precision = report['macro avg']['precision']
recall = report['macro avg']['recall']
f1_score = report['macro avg']['f1-score']
support = report['macro avg']['support']
accuracy = report['accuracy']

print("accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1_score)
print("support" , support)